<a href="https://colab.research.google.com/github/Destakka/102_ComposableLayout/blob/master/Code_CAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EfficientNetB1#

In [ ]:
# **bold text**
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import EfficientNetB1
import itertools
import os
import shutil
import random
import cv2
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import re, glob
import time

start_time = time.time()
im_size = 224

PATH = '/content/drive/MyDrive/Comvis/Dataset Comvis/Training_images'
dirs = os.listdir(PATH) #list directory in Land Use Images folder
label = 0
im_arr = []
lb_arr = []
X = []
y = []
for folder in dirs:
    folder_path = os.path.join(PATH, folder)  # Pastikan path benar
    if os.path.isdir(folder_path):  # Hanya proses jika path adalah folder
        count = 0
        for pic in glob.glob(folder_path + '/*'):  # Cari semua file di folder
            im = cv2.imread(pic)  # Baca gambar
            im = cv2.resize(im, (im_size, im_size))  # Resize ke 224x224
            im = np.array(im)  # Ubah jadi array
            count += 1
            X.append(im)  # Tambah ke array X
            y.append(label)  # Tambah label
            if count <= 7:  # Hanya untuk contoh
                im_arr.append({str(folder): im})
        print(f"Jumlah {folder}: {count}")
        label += 1
        lb_arr.append(folder)

# Konversi X dan y ke numpy array
X = np.array(X)
y = np.array(y)

print(f"--- {time.time() - start_time} seconds ---")
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
import matplotlib.pyplot as plt

# Visualisasi gambar dari im_arr
fig, axs = plt.subplots(2, 7, figsize=(20, 10))  # Grid 2x7 untuk 14 gambar
cnt = 0
for row in range(2):
    for col in range(7):
        if cnt < len(im_arr):  # Pastikan tidak melebihi jumlah gambar di im_arr
            for key, value in im_arr[cnt].items():
                axs[row, col].imshow(value)
                axs[row, col].set_title(key)
                axs[row, col].axis('off')  # Hilangkan axis
            cnt += 1
        else:
            axs[row, col].axis('off')  # Kosongkan plot jika tidak ada gambar
plt.tight_layout()
plt.show()
TEST_PATH = '/content/drive/MyDrive/Comvis/Dataset Comvis/Test_images'  # Ganti dengan path folder data test Anda
TRAIN_PATH = '/content/drive/MyDrive/Comvis/Dataset Comvis/Training_images'
VALID_PATH = '/content/drive/MyDrive/Comvis/Dataset Comvis/Validation_images' # Path untuk data validasi
#TEST_PATH = '/content/drive/MyDrive/BARU/Test/'

#X_test = []
#y_test = []
#label = 0

#for i in os.listdir(TEST_PATH):
    #for pic in glob.glob(TEST_PATH + i + '/*'):
        #im = cv2.imread(pic)
        #im = cv2.resize(im, (im_size, im_size))
        #im = np.array(im)
        #X_test.append(im)
        #y_test.append(label)
    #label = label + 1
def load_and_preprocess_data(data_path, im_size=224):
    X = []
    y = []
    label = 0
    for folder in os.listdir(data_path):  # Loop melalui folder di data_path
        folder_path = os.path.join(data_path, folder)
        if os.path.isdir(folder_path):  # Pastikan hanya memproses folder
            for pic in glob.glob(folder_path + '/*'):
                im = cv2.imread(pic)  # Baca gambar
                im = cv2.resize(im, (im_size, im_size))  # Resize ke im_size x im_size
                im = np.array(im)  # Ubah jadi array numpy
                X.append(im)  # Tambahkan gambar ke X
                y.append(label)  # Tambahkan label ke y
            label += 1  # Perbarui label untuk folder berikutnya
    return np.array(X), np.array(y)

# Contoh penggunaan
X_train, y_train = load_and_preprocess_data(TRAIN_PATH)
X_test, y_test = load_and_preprocess_data(TEST_PATH)
X_valid, y_valid = load_and_preprocess_data(VALID_PATH)

print(f"Training data shape: {X_train.shape}, Labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}, Labels shape: {y_test.shape}")
print(f"Validation data shape: {X_valid.shape}, Labels shape: {y_valid.shape}")
X_train, y_train = load_and_preprocess_data(TRAIN_PATH)
X_val, y_val = load_and_preprocess_data(VALID_PATH)
X_test, y_test = load_and_preprocess_data(TEST_PATH)
# PREPROCESSING
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rasio split data
train_ratio = 0.70
validation_ratio = 0.10
test_ratio = 0.20

# Split data menjadi train, validation, dan test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(1 - train_ratio), random_state=5)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(test_ratio / (test_ratio + validation_ratio)), random_state=1)

# Konversi tipe data ke float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

# Normalisasi data (ubah nilai menjadi antara 0 dan 1)
X_train /= 255.0
X_test /= 255.0
X_val /= 255.0

# One-hot encoding untuk label
y_train = to_categorical(y_train, num_classes=len(np.unique(y)))
y_val = to_categorical(y_val, num_classes=len(np.unique(y)))
y_test = to_categorical(y_test, num_classes=len(np.unique(y)))

# Data augmentation menggunakan ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

# Informasi data
print(f"Data Latih : {len(X_train)}")
print(f"Data Uji : {len(X_test)}")
print(f"Data Validasi : {len(X_val)}")
from tensorflow.keras.applications import ConvNeXtTiny # Import the ConvNeXtTiny class

# Load the ConvNeXtTiny model
efficientnet_model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))
#x = resnet_model.layers[-2].output
#dropOutCus = Dropout(0.5)(x)
#output = Dense(units=2, activation='softmax')(dropOutCus)

#efficientnet_model = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

x = efficientnet_model.output  # Output dari EfficientNetB1
x = Flatten()(x)  # Tambahkan lapisan Flatten untuk mengubah output 4D menjadi 2D
dropOutCus = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
output = Dense(units=2, activation='softmax')(dropOutCus)  # Output layer dengan 2 kelas dan aktivasi softmax
model = keras.Model(inputs=efficientnet_model.input, outputs=output)  # Buat model dengan EfficientNetB1

# Bekukan 100 lapisan pertama dari ConvNeXtTiny untuk transfer learning
for layer in efficientnet_model.layers[:50]:  # Bekukan 100 lapisan pertama EfficientNetB1
    layer.trainable = False
from tensorflow.keras.callbacks import LearningRateScheduler

# Fungsi scheduler untuk mengatur learning rate
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))  # Exponential decay setelah 10 epoch
lr_scheduler = LearningRateScheduler(scheduler, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# TRAINING
start_time = time.time()
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,  # Tingkatkan epoch untuk eksplorasi, EarlyStopping akan menghentikan pelatihan lebih awal
    batch_size=32,
    callbacks=[lr_scheduler]
)

# Evaluasi model
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy: {scores[1] * 100:.2f}%")
print(f"--- {time.time() - start_time:.2f} seconds ---")
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
import plotly.graph_objects as go
fig = go.Figure()
los = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['accuracy'],
                         mode='lines+markers',
                         name='Training accuracy'))
fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['val_accuracy'],
                         mode='lines+markers',
                         name='Validation accuracy'))
fig.update_layout(title='Accuracy',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))

los.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['loss'],
                         mode='lines+markers',
                         name='Training loss'))
los.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['val_loss'],
                         mode='lines+markers',
                         name='Validation loss'))
los.update_layout(title='Loss',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))

fig.show()
los.show()

# Evaluasi akurasi model
accuracy = scores[1] * 100  # Mengambil skor akurasi dari hasil evaluasi
print(f"Model Accuracy: {accuracy:.2f}%")
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Prediksi data uji
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(true_classes), yticklabels=np.unique(true_classes))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print("Classification Report:\n")
print(classification_report(true_classes, predicted_classes))


#ConvNextTiny#


In [ ]:
# **bold text**
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import EfficientNetB1
import itertools
import os
import shutil
import random
import cv2
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import re, glob
import time

start_time = time.time()
im_size = 224

PATH = '/content/drive/MyDrive/Comvis/Dataset Comvis/Training_images'
dirs = os.listdir(PATH) #list directory in Land Use Images folder
label = 0
im_arr = []
lb_arr = []
X = []
y = []
for folder in dirs:
    folder_path = os.path.join(PATH, folder)  # Pastikan path benar
    if os.path.isdir(folder_path):  # Hanya proses jika path adalah folder
        count = 0
        for pic in glob.glob(folder_path + '/*'):  # Cari semua file di folder
            im = cv2.imread(pic)  # Baca gambar
            im = cv2.resize(im, (im_size, im_size))  # Resize ke 224x224
            im = np.array(im)  # Ubah jadi array
            count += 1
            X.append(im)  # Tambah ke array X
            y.append(label)  # Tambah label
            if count <= 7:  # Hanya untuk contoh
                im_arr.append({str(folder): im})
        print(f"Jumlah {folder}: {count}")
        label += 1
        lb_arr.append(folder)

# Konversi X dan y ke numpy array
X = np.array(X)
y = np.array(y)

print(f"--- {time.time() - start_time} seconds ---")
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")
import matplotlib.pyplot as plt

# Visualisasi gambar dari im_arr
fig, axs = plt.subplots(2, 7, figsize=(20, 10))  # Grid 2x7 untuk 14 gambar
cnt = 0
for row in range(2):
    for col in range(7):
        if cnt < len(im_arr):  # Pastikan tidak melebihi jumlah gambar di im_arr
            for key, value in im_arr[cnt].items():
                axs[row, col].imshow(value)
                axs[row, col].set_title(key)
                axs[row, col].axis('off')  # Hilangkan axis
            cnt += 1
        else:
            axs[row, col].axis('off')  # Kosongkan plot jika tidak ada gambar
plt.tight_layout()
plt.show()
TEST_PATH = '/content/drive/MyDrive/Comvis/Dataset Comvis/Test_images'  # Ganti dengan path folder data test Anda
TRAIN_PATH = '/content/drive/MyDrive/Comvis/Dataset Comvis/Training_images'
VALID_PATH = '/content/drive/MyDrive/Comvis/Dataset Comvis/Validation_images' # Path untuk data validasi
#TEST_PATH = '/content/drive/MyDrive/BARU/Test/'

#X_test = []
#y_test = []
#label = 0

#for i in os.listdir(TEST_PATH):
    #for pic in glob.glob(TEST_PATH + i + '/*'):
        #im = cv2.imread(pic)
        #im = cv2.resize(im, (im_size, im_size))
        #im = np.array(im)
        #X_test.append(im)
        #y_test.append(label)
    #label = label + 1
def load_and_preprocess_data(data_path, im_size=224):
    X = []
    y = []
    label = 0
    for folder in os.listdir(data_path):  # Loop melalui folder di data_path
        folder_path = os.path.join(data_path, folder)
        if os.path.isdir(folder_path):  # Pastikan hanya memproses folder
            for pic in glob.glob(folder_path + '/*'):
                im = cv2.imread(pic)  # Baca gambar
                im = cv2.resize(im, (im_size, im_size))  # Resize ke im_size x im_size
                im = np.array(im)  # Ubah jadi array numpy
                X.append(im)  # Tambahkan gambar ke X
                y.append(label)  # Tambahkan label ke y
            label += 1  # Perbarui label untuk folder berikutnya
    return np.array(X), np.array(y)

# Contoh penggunaan
X_train, y_train = load_and_preprocess_data(TRAIN_PATH)
X_test, y_test = load_and_preprocess_data(TEST_PATH)
X_valid, y_valid = load_and_preprocess_data(VALID_PATH)

print(f"Training data shape: {X_train.shape}, Labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}, Labels shape: {y_test.shape}")
print(f"Validation data shape: {X_valid.shape}, Labels shape: {y_valid.shape}")
X_train, y_train = load_and_preprocess_data(TRAIN_PATH)
X_val, y_val = load_and_preprocess_data(VALID_PATH)
X_test, y_test = load_and_preprocess_data(TEST_PATH)
# PREPROCESSING
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rasio split data
train_ratio = 0.70
validation_ratio = 0.10
test_ratio = 0.20

# Split data menjadi train, validation, dan test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(1 - train_ratio), random_state=5)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(test_ratio / (test_ratio + validation_ratio)), random_state=1)

# Konversi tipe data ke float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

# Normalisasi data (ubah nilai menjadi antara 0 dan 1)
X_train /= 255.0
X_test /= 255.0
X_val /= 255.0

# One-hot encoding untuk label
y_train = to_categorical(y_train, num_classes=len(np.unique(y)))
y_val = to_categorical(y_val, num_classes=len(np.unique(y)))
y_test = to_categorical(y_test, num_classes=len(np.unique(y)))

# Data augmentation menggunakan ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

# Informasi data
print(f"Data Latih : {len(X_train)}")
print(f"Data Uji : {len(X_test)}")
print(f"Data Validasi : {len(X_val)}")
from tensorflow.keras.applications import ConvNeXtTiny # Import the ConvNeXtTiny class

# Load the ConvNeXtTiny model
convnext_model = ConvNeXtTiny(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))
#x = resnet_model.layers[-2].output
#dropOutCus = Dropout(0.5)(x)
#output = Dense(units=2, activation='softmax')(dropOutCus)

#efficientnet_model = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

x = convnext_model.output  # Output dari EfficientNetB1
x = Flatten()(x)  # Tambahkan lapisan Flatten untuk mengubah output 4D menjadi 2D
dropOutCus = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
output = Dense(units=2, activation='softmax')(dropOutCus)  # Output layer dengan 2 kelas dan aktivasi softmax
model = keras.Model(inputs=convnext_model.input, outputs=output)  # Buat model dengan EfficientNetB1

# Bekukan 100 lapisan pertama dari ConvNeXtTiny untuk transfer learning
for layer in convnext_model.layers[:50]:  # Bekukan 100 lapisan pertama EfficientNetB1
    layer.trainable = False
from tensorflow.keras.callbacks import LearningRateScheduler

# Fungsi scheduler untuk mengatur learning rate
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))  # Exponential decay setelah 10 epoch
lr_scheduler = LearningRateScheduler(scheduler, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# TRAINING
start_time = time.time()
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,  # Tingkatkan epoch untuk eksplorasi, EarlyStopping akan menghentikan pelatihan lebih awal
    batch_size=32,
    callbacks=[lr_scheduler]
)

# Evaluasi model
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy: {scores[1] * 100:.2f}%")
print(f"--- {time.time() - start_time:.2f} seconds ---")
from matplotlib import pyplot as plt

plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
import plotly.graph_objects as go
fig = go.Figure()
los = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['accuracy'],
                         mode='lines+markers',
                         name='Training accuracy'))
fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['val_accuracy'],
                         mode='lines+markers',
                         name='Validation accuracy'))
fig.update_layout(title='Accuracy',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))

los.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['loss'],
                         mode='lines+markers',
                         name='Training loss'))
los.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['val_loss'],
                         mode='lines+markers',
                         name='Validation loss'))
los.update_layout(title='Loss',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))

fig.show()
los.show()

# Evaluasi akurasi model
accuracy = scores[1] * 100  # Mengambil skor akurasi dari hasil evaluasi
print(f"Model Accuracy: {accuracy:.2f}%")
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Prediksi data uji
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(true_classes), yticklabels=np.unique(true_classes))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print("Classification Report:\n")
print(classification_report(true_classes, predicted_classes))
